# Import modules

In [1]:
import gdal
import shapefile
from osgeo import osr
import glob
from scipy import interpolate
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame
import numpy as np

In [2]:
floodmap=gdal.Open('./data/Ambiental_DataJavelin_FloodMap_QA/Domain_Errors/CombinedOutput_FloodMap/NOR_Fluvial_100yr.tif')

In [3]:
arr = floodmap.ReadAsArray()

In [4]:
proj_wkt = floodmap.GetProjection()


In [5]:
proj = osr.SpatialReference()
proj.ImportFromWkt(proj_wkt)


0

In [6]:
geo_transform = floodmap.GetGeoTransform()

In [7]:
origin_x = geo_transform[0]
origin_y = geo_transform[3]
pixel_width = geo_transform[1]
pixel_height = geo_transform[5]
n_cols = floodmap.RasterXSize
n_rows = floodmap.RasterYSize
n_cols, n_rows

(23712, 29062)

In [8]:
band = floodmap.GetRasterBand(1)
gdal.GetDataTypeName(band.DataType)

'Float32'

In [9]:
extent_lonlat = (
    origin_x, 
    origin_x + (pixel_width * floodmap.RasterXSize),
    origin_y + (pixel_height * floodmap.RasterYSize),
    origin_y
)

## Idea 3
* Read in shapefile
* Match points and tangent to image
* plot difference across boundary

In [10]:
def find_nearest(array, value):
    ''' Find nearest value is an array '''
    idx=np.empty((value.size))
    for i in range(0,value.size):
        idx[i] = (np.abs(array-value[i])).argmin()
    return idx.astype('int')

In [11]:
sfiles=glob.glob('./data/Ambiental_DataJavelin_FloodMap_QA/Domain_Errors/Domains/*.shp')

In [16]:
df=[]
for ii, s in enumerate(sfiles[0:100]):
    sf = shapefile.Reader(s)
    sh_points=np.array(sf.shapes()[0].points)
    xpix=np.arange(extent_lonlat[0],extent_lonlat[1],pixel_width)
    ypix=np.arange(extent_lonlat[3],extent_lonlat[2],pixel_height)
    
# loop over shape points
    points=[]
    pix_dist=np.array([3,4])

    for i in range(0,sh_points.shape[0]-1):        
        #need to interpolate between points in shapefile
        fx = interpolate.interp1d(sh_points[i:i+2,0],sh_points[i:i+2,1],kind='slinear',fill_value='extrapolate')
        if sh_points[i,0]<sh_points[i+1,0]:
            xnew=np.arange(sh_points[i,0],sh_points[i+1,0],pixel_width)
        else:
            xnew=np.arange(sh_points[i,0],sh_points[i+1,0],-pixel_width)
        ynew=fx(xnew)
        fy=interpolate.interp1d(sh_points[i:i+2,1],sh_points[i:i+2,0],kind='slinear',fill_value='extrapolate')
        if sh_points[i,1]>sh_points[i+1,1]:
            ynew_2=np.arange(sh_points[i,1],sh_points[i+1,1],pixel_height)
        else:
            ynew_2=np.arange(sh_points[i,1],sh_points[i+1,1],-pixel_height)
        xnew_2=fy(ynew_2)
        
        for j in range(xnew.shape[0]-1):
            try:
                #calculate difference in x and y for tangent
                dy=ynew[j+1]-ynew[j]
                dx=xnew[j+1]-xnew[j]
                angle=0.5*np.pi-np.arctan(dy/dx)
                t_x=pix_dist*pixel_width*np.cos(angle)#ideally should be 0.5*pixel but more stable if 1.1
                t_y=pix_dist*pixel_height*np.sin(angle)
                #inner value
                x_in=find_nearest(xpix,xnew[j]+t_x)
                y_in=find_nearest(ypix,ynew[j]+t_y)
                #outer value
                x_out=find_nearest(xpix,xnew[j]-t_x)
                y_out=find_nearest(ypix,ynew[j]-t_y)
            
                points.append([np.abs(np.max(arr[y_in,x_in])-np.max(arr[y_out,x_out])),xnew[j],ynew[j]])
            except(IndexError):
                print('issue')
        for j in range(xnew_2.shape[0]-1):
            #calculate difference in x and y for tangent
            dy=ynew_2[j+1]-ynew_2[j]
            dx=xnew_2[j+1]-xnew_2[j]
            angle=0.5*np.pi-np.arctan(dy/dx)
            t_x=pix_dist*pixel_width*np.cos(angle)#ideally should be 0.5*pixel but more stable if 1.1
            t_y=pix_dist*pixel_height*np.sin(angle)
            #inner value
            x_in=find_nearest(xpix,xnew_2[j]+t_x)
            y_in=find_nearest(ypix,ynew_2[j]+t_y)
        
            #outer value
            x_out=find_nearest(xpix,xnew_2[j]-t_x)
            y_out=find_nearest(ypix,ynew_2[j]-t_y)
            points.append([np.abs(np.max(arr[y_in,x_in])-np.max(arr[y_out,x_out])),xnew_2[j],ynew_2[j]])
    points=np.array(points)
    ind=points[:,0]>100.0
    data = {'Domain': s.split('/')[-1] * ind.sum(),
        'lat': points[ind,2],
        'lon': points[ind,1]}

    df.append(pd.DataFrame(data))
    print('Domain ',s.split('/')[-1], ii,len(sfiles))
alldf = pd.concat(df)
geometry = [Point(xy) for xy in zip(alldf['lon'],alldf['lat'])]
gdf = GeoDataFrame(alldf, geometry=geometry)
gdf.to_file(filename='./all_points_pixspread')

/Users/pdh21/anaconda3/envs/new/lib/python3.6/site-packages/ipykernel/__main__.py:49: RuntimeWarning: divide by zero encountered in double_scalars


Domain  NOR_1000_1_Domain.shp 0 5151
Domain  NOR_1001_1_Domain.shp 1 5151
Domain  NOR_1002_1_Domain.shp 2 5151
Domain  NOR_1003_1_Domain.shp 3 5151
Domain  NOR_1004_1_Domain.shp 4 5151
Domain  NOR_1005_1_Domain.shp 5 5151
Domain  NOR_1006_1_Domain.shp 6 5151
Domain  NOR_1007_1_Domain.shp 7 5151
Domain  NOR_1008_1_Domain.shp 8 5151
Domain  NOR_1009_1_Domain.shp 9 5151
Domain  NOR_100_1_Domain.shp 10 5151
Domain  NOR_1010_1_Domain.shp 11 5151
Domain  NOR_1011_1_Domain.shp 12 5151
Domain  NOR_1012_1_Domain.shp 13 5151
Domain  NOR_1013_1_Domain.shp 14 5151
Domain  NOR_1014_1_Domain.shp 15 5151
Domain  NOR_1015_1_Domain.shp 16 5151
Domain  NOR_1016_1_Domain.shp 17 5151
Domain  NOR_1017_1_Domain.shp 18 5151
Domain  NOR_1018_1_Domain.shp 19 5151
Domain  NOR_1019_1_Domain.shp 20 5151
Domain  NOR_101_1_Domain.shp 21 5151
Domain  NOR_1020_1_Domain.shp 22 5151
Domain  NOR_1021_1_Domain.shp 23 5151
Domain  NOR_1022_1_Domain.shp 24 5151
Domain  NOR_1023_1_Domain.shp 25 5151
Domain  NOR_1024_1_Domai